In [1]:
%%html
<style>
table {
    display: inline-block
}
</style>

# 使用FLASK创建应用：个人主页
> <img style="width:200px; display: inline-block" src="https://resource.xtalker.cn:8000/_static/img/dog-smile.png"/>

        发布个人主页，介绍个人履历


# 1.概述

## 1.1.基本概念
### SOA
        SOA（Service-Oriented Architecture，面向服务的架构）是一种软件设计架构模式，它将应用程序的不同功能单元（称为服务）通过这些服务之间定义良好的接口和契约联系起来。接口是采用中立的方式进行定义的，它应该独立于实现服务的硬件平台、操作系统和编程语言。这使得构建在各种各样的系统中的服务可以以一种统一和通用的方式进行交互。
        以上是对SOA的字面解释，不理解没有关系。我们以目前开发实例中关于表现逻辑和业务逻辑分离来理解以上的思想。在WEB开发中，表现逻辑和业务逻辑是两个重要的组成部分，它们各自承担不同的角色和功能。以下是关于这两者的详细解释：
        
### 表现逻辑
    表现逻辑（Presentation Logic）
        表现逻辑主要负责处理与用户界面（UI）相关的交互和显示。这包括页面布局、元素样式、用户输入验证以及根据用户输入或系统状态更新UI等。
    简单来说就是你写的哪些HTML/CSS/JS等代码。其重要特征是，逻辑是在客户端的浏览器上执行的。

### 业务逻辑
    业务逻辑（Business Logic）
        业务逻辑是系统处理业务规则和数据的关键部分。它负责处理用户的请求，根据业务规则对数据进行验证、计算或转换，并将处理结果返回给表现层或持久层。
    简单来说就是你写的哪些Python代码以及服务端的一些配置代码等。其重要特征是，逻辑是在服务端执行的。

## 1.2.SOA设计思想的演进过程

### 表现逻辑和业务逻辑不分家
> <img style="width:50%; display: inline-block" src="https://resource.xtalker.cn:8000/_static/img/soa-architecture1.png"/>

    用户通过WEB浏览器访问应用服务器，应用服务器的数据使用数据库进行存储。这是一个典型的系统架构。
    优势：
        适合个人或者项目人数非常少的情况，系统结构简单
    缺点：
        不适合不同技术栈的开发人员共同开发项目
        由于各种逻辑处理混杂在一起，系统水平扩展和容错设计会变得困难
### 贴士
    上一节课的实现方式即这种模式，如下所示的代码中，表现逻辑和业务逻辑并没有什么区分。

```python
@app.route("/")
def hello():
    return '''
Hello, World!
I am lihua.
[from http://hello.world]
    '''
```
        
### 表现逻辑和业务逻辑分离：采用模板实现
> <img style="width:50%; display: inline-block" src="https://resource.xtalker.cn:8000/_static/img/soa-architecture2.png"/>

    使用HTML模板来实现表现逻辑，模板中嵌入业务逻辑单元实现业务和表现的结合。这种方式使得页面布局和业务实现的逻辑分开处理，更加易于维护。例如下的代码中表现逻辑放在了 index.html 中，业务逻辑处理结果通过变量传递给模板。

```python
@app.route("/")
def home():
    count = get_hit_count()
    # 传递变量给渲染模版：模版文件位于 code/templates/index.html
    return render_template("index.html",hits=count)

```

### 表现逻辑和业务逻辑彻底分离：采用js-websocket等技术实现
> <img style="width:70%; display: inline-block" src="https://resource.xtalker.cn:8000/_static/img/soa-architecture3.png"/>

    使用JavaScript实现页面的动态更新，使得表现和业务逻辑的彻底分离。一般需要较为高超的Web客户端编程能力，同时降低服务端编程的难度。由于JS的开发本次课程不再涉及。

## 1.3.本次课程实验内容
    实现一个个人主页，要求使用：
    1.Flask Jinja 模板引擎 实现个人主页要求有文字和图片等
    2.Redis 缓存机制，实现页面访问计数器功能


# 2.code
> 完整代码：[下载](https://resource.xtalker.cn:8000/_static/code/flask_resume.zip)

    创建自己的项目目录(任何你喜欢的目录名均可，以后就是 Docker-Compose的项目名)：
        flask_resume
    在你的项目目录下准备代码文件：
        code/app.py
        code/templates/index.html
        code/static/dog.png
        requirements.txt
        Dockerfile
        compose.yml
        pip.conf
    系统配置（不是必须）：
        Windows默认位置：C:\Windows\System32\drivers\etc\hosts
        Linux：/etc/hosts

### app.py
    代码解析
```python
import time
# Redis 数据库
import redis
# 渲染模版：render_template
from flask import Flask, render_template, request
app = Flask(__name__)

# 启用调试模式
if __name__ == '__main__':
    app.run(debug=True)

# Redis作为应用缓存用于记录网站访问次数
cache = redis.Redis(host='redis', port=6379)

def get_hit_count():
    retries = 5
    while True:
        try:
            return cache.incr('hits')
        except redis.exceptions.ConnectionError as exc:
            if retries == 0:
                raise exc
            retries -= 1
            time.sleep(0.5)

# 用户访问根目录
@app.route("/")
def home():
    count = get_hit_count()
    # 传递变量给渲染模版：模版文件位于 code/templates/index.html
    return render_template("index.html",hits=count)
```

### 模版文件：code/templates/index.html
    flask 会到默认的模版文件目录 templates 下寻找
    index.html代码片段如下：
```html
<span style='font-size:20.0pt;mso-bidi-font-size:
16.0pt;font-family:宋体'>单身狗的履历<img style="width: 100px; display: inline-block" src="static/dog.png"/> 查阅次数：（{{ hits }}）次<span lang=EN-US><o:p></o:p></span></span>
```

    1.计数器
        查阅次数：（{{ hits }}）次
        与 app.py 模版调用函数中的变量：render_template("index.html",hits=count) 中的参数对应
    2.图片链接
        <img style="width: 100px; display: inline-block" src="static/dog.png"/>
        图片文件作为静态文件按照习惯放在 code/static 目录下
    

### requirements.txt

```
flask
redis
```

### Dockerfile

```shell
FROM python:alpine
WORKDIR /code
COPY requirements.txt requirements.txt
# 部署pip.conf文件，用于加速pip安装过程
COPY pip.conf /etc/pip.conf
RUN pip install -r requirements.txt
CMD ["flask", "run"]

```

### compose.yml

```yaml
services:
  # python-flask容器实例
  web:
    image: resume:alpine
    build: .
    restart: "unless-stopped"
    tty: true
    networks:
      - python-net
    hostname: web
    ports:
     - "80:5000"
    environment:
      - FLASK_APP=app.py
      - FLASK_RUN_HOST=0.0.0.0
    volumes:
      # 将目录映射到容器，包含代码、模版和图片资源
      - .\code:/code
    depends_on:
      - redis
  
  # Redis数据容器实例
  redis:
    image: redis:alpine
    networks:
      - python-net

networks:
  python-net:
    name: python-br
```

### hosts
    hosts:
        C:\Windows\System32\drivers\etc\hosts
    用文本编辑器以管理员模式打开hosts文件，在文件的结尾处增加一条记录，如下所示：
    
```ini
# 你也可以使用任何你喜欢的域名来替代 resume.dog
127.0.0.1 resume.dog www.resume.dog
```
    测试一下是否生效：
```shell
ping resume.dog
# 生效，输出如下：
正在 Ping hello.world [127.0.0.1] 具有 32 字节的数据:
来自 127.0.0.1 的回复: 字节=32 时间<1ms TTL=128
来自 127.0.0.1 的回复: 字节=32 时间<1ms TTL=128
来自 127.0.0.1 的回复: 字节=32 时间<1ms TTL=128
# 未生效
Ping 请求找不到主机 resume.dog。请检查该名称，换一个域名然后重试。
```

# 3.test

## 命令执行过程

```shell
cd flask_resume

# 构建
docker-compose build
[+] Building 0.3s (10/10) FINISHED                                                                       docker:default
 => [web internal] load build definition from Dockerfile                                                           0.0s
 => => transferring dockerfile: 256B                                                                               0.0s
 => [web internal] load metadata for docker.io/library/python:alpine                                               0.0s
 => [web internal] load .dockerignore                                                                              0.0s
 => => transferring context: 2B                                                                                    0.0s
 => [web 1/5] FROM docker.io/library/python:alpine                                                                 0.0s
 => [web internal] load build context                                                                              0.0s
 => => transferring context: 65B                                                                                   0.0s
 => CACHED [web 2/5] WORKDIR /code                                                                                 0.0s
 => CACHED [web 3/5] COPY requirements.txt requirements.txt                                                        0.0s
 => CACHED [web 4/5] COPY pip.conf /etc/pip.conf                                                                   0.0s
 => CACHED [web 5/5] RUN pip install -r requirements.txt                                                           0.0s
 => [web] exporting to image                                                                                       0.0s
 => => exporting layers                                                                                            0.0s
 => => writing image sha256:eaf643ee1167b1f4c5d26328f6833284944dd9324129ff34c7d18057cc37c6c9                       0.0s
 => => naming to docker.io/library/resume:alpine                                                                   0.0s

# 创建并启动
docker-compose up -d
[+] Running 3/3
 ✔ Network python-br               Created                                                                         0.1s
 ✔ Container flask_resume-redis-1  Started                                                                         0.2s
 ✔ Container flask_resume-web-1    Started

# 构建成功会在主机生成镜像：resume:alpine
docker-compose images
CONTAINER              REPOSITORY          TAG                 IMAGE ID            SIZE
flask_resume-redis-1   redis               alpine              f597a450f464        40.7MB
flask_resume-web-1     resume              alpine              eaf643ee1167        69.4MB
```

## 浏览器访问 http://resume.dog
> <img style="width:60%; display: inline-block" src="https://resource.xtalker.cn:8000/_static/img/flask-resume-demo.png"/>


# 4.课后复习预习
    1.了解 Redis 缓存机制
    2.了解 Python 如何调用其它程序